In [1]:
# ===================================================================
# PASO 1: CONFIGURACIÓN E INSTALACIÓN DE LIBRERÍAS
# ===================================================================

import pandas as pd
import requests
import sqlite3

print("✅ Librerías importadas.")

# ===================================================================
# PASO 2: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente, lo que es más eficiente que bajar un CSV.
# Aumentamos el límite para traer más filas (ajusta si es necesario).
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla (ej: 404)
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame() # Creamos un dataframe vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()

✅ Librerías importadas.
📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [2]:
df_raw.columns

Index(['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento',
       'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16',
       'tasa_matriculaci_n_5_16', 'cobertura_neta',
       'cobertura_neta_transici_n', 'cobertura_neta_primaria',
       'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta',
       'cobertura_bruta_transici_n', 'cobertura_bruta_primaria',
       'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n',
       'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
       'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n',
       'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media',
       'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria',
       'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia',
       'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
       'repitencia_media', 'tama_o_promedio_de_grupo',
       'sedes_conectadas_a_internet'],
   

# Punto 1

Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?

In [3]:
# Filtrar registros válidos (evitar registros nacionales o nulos)
df_filtrado = df_raw[
    (df_raw['departamento'] != 'NACIONAL') &
    (df_raw['municipio'].notna()) &
    (df_raw['tasa_matriculaci_n_5_16'].notna())
]

# Asegurar tipo numérico
df_filtrado['tasa_matriculaci_n_5_16'] = df_filtrado['tasa_matriculaci_n_5_16'].astype(float)

# Agrupar por municipio y año (si existe columna de año)
if 'a_o' in df_filtrado.columns:
    escolaridad_mpio = (
        df_filtrado
        .groupby(['departamento', 'municipio', 'a_o'])['tasa_matriculaci_n_5_16']
        .mean()
        .reset_index()
        .rename(columns={'tasa_matriculaci_n_5_16': 'porcentaje_escolaridad'})
    )
else:
    escolaridad_mpio = (
        df_filtrado
        .groupby(['departamento', 'municipio'])['tasa_matriculaci_n_5_16']
        .mean()
        .reset_index()
        .rename(columns={'tasa_matriculaci_n_5_16': 'porcentaje_escolaridad'})
    )

# Mostrar resultados
escolaridad_mpio.sort_values(by='porcentaje_escolaridad', ascending=False).head(10)


/tmp/ipython-input-3-596449628.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['tasa_matriculaci_n_5_16'] = df_filtrado['tasa_matriculaci_n_5_16'].astype(float)


,departamento,municipio,a_o,porcentaje_escolaridad
2035,Atlántico,Puerto Colombia,2018,279.03
2034,Atlántico,Puerto Colombia,2017,273.80
2033,Atlántico,Puerto Colombia,2016,266.50
6682,Cundinamarca,Cota,2017,240.70
6683,Cundinamarca,Cota,2018,239.34
6681,Cundinamarca,Cota,2016,236.70
6687,Cundinamarca,Cota,2022,197.50
6684,Cundinamarca,Cota,2019,195.16
12585,Santander,Sabana de Torres,2018,194.87
6685,Cundinamarca,Cota,2020,193.34


# Punto 2
2.¿Cómo compararía el rendimiento educativo por municipios?


En términos de desempeño educativo aproximado, medido a partir de la tasa promedio de matrícula de niños entre 5 y 16 años como un proxy del compromiso del sistema educativo, los municipios de Cundinamarca destacan notablemente. El municipio de Cota lidera el ranking con un valor promedio de 209.95, seguido por Puerto Colombia en Atlántico, con 195.60. Otros municipios cundinamarqueses como Tenjo (149.60), La Calera (144.08) y Fúquene (128.59) también figuran entre los primeros lugares, reflejando un alto nivel de cobertura escolar en la región.

Además, municipios como Salento en el departamento del Quindío (139.27), Restrepo y Puerto Gaitán en Meta (134.68 y 129.72 respectivamente), y Sabana de Torres en Santander (132.08), también presentan tasas destacadas.

In [5]:
df_comparacion = (
    df_raw
    .query("departamento != 'NACIONAL' and tasa_matriculaci_n_5_16.notna()")
    .copy()
)

df_comparacion['tasa_matriculaci_n_5_16'] = df_comparacion['tasa_matriculaci_n_5_16'].astype(float)

# Agrupamos por municipio y promediamos la tasa
df_mpio_tasa = (
    df_comparacion
    .groupby(['departamento', 'municipio'])['tasa_matriculaci_n_5_16']
    .mean()
    .reset_index()
    .rename(columns={'tasa_matriculaci_n_5_16': 'proxy_rendimiento'})
)

# Visualizamos los municipios con mayor "proxy de rendimiento"
df_mpio_tasa.sort_values('proxy_rendimiento', ascending=False).head(10)


,departamento,municipio,proxy_rendimiento
521,Cundinamarca,Cota,209.951250
160,Atlántico,Puerto Colombia,195.596250
595,Cundinamarca,Tenjo,149.599000
548,Cundinamarca,La Calera,144.078462
898,Quindio,Salento,139.272308
762,Meta,Restrepo,134.677500
979,Santander,Sabana de Torres,132.083750
758,Meta,Puerto Gaitán,129.721000
531,Cundinamarca,Fúquene,128.590769
587,Cundinamarca,Subachoque,127.242308


# Punto 3
¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

In [6]:
import sqlite3

# Conexión a una base en memoria
conn = sqlite3.connect(":memory:")

# Asegurar que el campo es numérico
df_raw['tasa_matriculaci_n_5_16'] = pd.to_numeric(df_raw['tasa_matriculaci_n_5_16'], errors='coerce')

# Filtrar valores válidos y excluir el registro 'NACIONAL'
df_sql = df_raw.query("departamento != 'NACIONAL' and tasa_matriculaci_n_5_16.notna()")

# Subir el DataFrame a SQLite
df_sql.to_sql("matricula", conn, index=False, if_exists='replace')

# Ejecutar SQL: promedio por departamento
query = """
SELECT
    departamento,
    AVG(tasa_matriculaci_n_5_16) AS cobertura_promedio
FROM matricula
GROUP BY departamento
ORDER BY cobertura_promedio DESC
LIMIT 10;
"""

top_departamentos = pd.read_sql(query, conn)
top_departamentos


,departamento,cobertura_promedio
0,Bogotá D.C.,96.410000
1,Cesar,96.113944
2,Sucre,95.251538
3,Magdalena,94.591410
4,Quindio,93.880000
5,Meta,91.258147
6,Atlántico,89.455816
7,Casanare,89.254321
8,Córdoba,88.003057
9,Tolima,87.923552
